## Pytorch方案

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorboardX import SummaryWriter

In [2]:
#维度和大小参数定义
batch_size = 600
input_dim = 1000
output_dim = 1
hidden_dim1 = 200
hidden_dim2 = 50
# 数据虚拟 (x,y)
X = np.random.randn(input_dim,batch_size)
W = np.random.randn(output_dim,input_dim)
y = np.dot(W,X) + np.random.randn(1,batch_size)/10000 #添加偏差项
X_t = torch.tensor(X.T).float()
y_t = torch.tensor(y.T).float()


In [3]:
X_t.shape,y_t.shape

(torch.Size([600, 1000]), torch.Size([600, 1]))

In [4]:
class Net(torch.nn.Module):  # 继承 torch 的 Module
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()     # 继承 __init__ 功能
        # 定义每层用什么样的形式
        self.hidden1 = nn.Linear(input_dim,hidden_dim1)   # 隐藏层线性输出1000*200
        self.hidden2 = nn.Linear(hidden_dim1,hidden_dim2)
        self.predict = nn.Linear(hidden_dim2,output_dim)   # 输出层线性输出 200*1       

    def forward(self, x):   # 这同时也是 Module 中的 forward 功能
        # 正向传播输入值, 神经网络分析出输出值
        x = F.relu(self.hidden1(x))      # 激励函数(隐藏层的线性值)
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # 输出值
        return x

In [5]:
net = Net(n_feature=input_dim, n_hidden=hidden_dim1, n_output=output_dim)
writer = SummaryWriter('D:/python_code/Pytorch')
writer_loss = SummaryWriter('D:/python_code/Pytorch/test')

with writer:
    writer.add_graph(net, (X_t,))
print(net)

Net(
  (hidden1): Linear(in_features=1000, out_features=200, bias=True)
  (hidden2): Linear(in_features=200, out_features=50, bias=True)
  (predict): Linear(in_features=50, out_features=1, bias=True)
)


In [6]:
criterion=nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.002)  # 传入 net的所有参数,学习率
loss_func = torch.nn.MSELoss() # 预测值和真实值的误差计算公式 (均方差)

In [7]:
for t in range(0,1000):
    prediction = net(Variable(X_t))     # 喂给 net 训练数据 x, 输出预测值
    loss = loss_func(prediction, Variable(y_t))     # 计算两者的误差
    writer_loss.add_scalar("loss",loss,global_step=t, walltime=None)#将我们所需要的数据保存在文件里面供可视化使用
    optimizer.zero_grad()   # 清空上一步的残余更新参数值
    loss.backward()         # 误差反向传播, 计算参数更新值
    optimizer.step()        # 将参数更新值施加到 net 的 parameters 上
    writer_loss.close()
    if loss<1e-4:
        break

68


In [8]:
loss

tensor(8.2762e-05, grad_fn=<MseLossBackward>)